# Weather data fetch tests
This notebook will walk through how to get forecasted weather data for a location. 

the weather api used is from weather.gov: https://www.weather.gov/documentation/services-web-api

# the weather fetching process is:
1.  get the lat long for the sites
2.  get the grid points
3.  call the for the appropriate forecast 
4.  convert the json payload into a dataframe 
5.  put it all together


# 1 Get lat long for each site
Using google maps, I typed in the locations and then asked for the lat and long of that spot on the map. Here are the lat long coordinates for our locations
![caltech](caltech_lat_long.png)

In [19]:
import requests
import pandas as pd
import json
from typing import Dict

caltech_lat =   34.134785646454844
caltech_lon = -118.11691382579643

jpl_lat = 34.20142342818471
jpl_lon = -118.17126565774107

office_lat = 37.33680466796926
office_lon = -121.90743423142634

# 2 Get grid points
The way the weather api system works is by finding the station's grid points that are closest to the lat long coordinates. The code below gets the grid x and grid y from the lat long coordinates

In [2]:
def get_grid_points(latitude, longitude):
    url = f'https://api.weather.gov/points/{latitude},{longitude}'
    print(url)
    
    r = requests.get(url)
    
    print('status code:', r.status_code)
    payload = r.json()
    
    gridId = payload['properties']['gridId']
    gridX = payload['properties']['gridX']
    gridY = r.json()['properties']['gridY']
    return gridId, gridX, gridY
cal = get_grid_points(caltech_lat, caltech_lon)
off = get_grid_points(office_lat, office_lon)

https://api.weather.gov/points/34.134785646454844,-118.11691382579643
status code: 200
https://api.weather.gov/points/37.33680466796926,-121.90743423142634
status code: 200


In [3]:
ct_grid, ct_grid_x, ct_grid_y = get_grid_points(caltech_lat, caltech_lon)
jpl_grid, jpl_grid_x, jpl_grid_y = get_grid_points(jpl_lat, jpl_lon)
office_grid, office_grid_x, office_grid_y = get_grid_points(office_lat, office_lon)

https://api.weather.gov/points/34.134785646454844,-118.11691382579643
status code: 200
https://api.weather.gov/points/34.20142342818471,-118.17126565774107
status code: 200
https://api.weather.gov/points/37.33680466796926,-121.90743423142634
status code: 200


# 3 get forecast
Once we have the office, gridx and gridy for the location, we can get three different kinds of weather forecasts

    forecast - forecast for 12h periods over the next seven days
    forecastHourly - forecast for hourly periods over the next seven days
    forecastGridData - raw forecast data over the next seven days



In [18]:
'''
def get_weather_forecast(office, gridX, gridY, forecast_type='forecast'):
    if forecast_type not in ['12_hour', '7day_hourly', '7day_grid']:
        print(f'{forecast_type} is not a valid option. please select 12_hour, 7day_hourly, or 7day_grid', )
        pass
    
    forecast_urls = {'12_hour':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast',
                     '7day_hourly':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/hourly',
                     '7day_grid':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}'}
    
    url = forecast_urls[forecast_type]
    print(url)
    r = requests.get(url)
    return r.json()
'''
type(ct_grid_x)

int

In [20]:
def get_weather_forecast(office:str, gridX:int, gridY:int)->Dict:
    """forecast for 12h periods over the next seven days
    args:
        office: str - the 3 letter code for the weather station
        gridX: int - the X coordinate for the weather grid
        gridY: int - the Y coordinate for the weather grid
    returns:
        dict - the json payload
    """
    url = f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast'
    print(url)
    r = requests.get(url, headers={'Accept': 'application/geo+json'})
    return r.json()

In [21]:
def get_hourly_weather_forecast(office:str, gridX:int, gridY:int) -> Dict:
    """forecast for hourly periods over the next seven days
    args:
        office: str - the 3 letter code for the weather station
        gridX: int - the X coordinate for the weather grid
        gridY: int - the Y coordinate for the weather grid
    returns:
        dict - the json payload
    """
    url = f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/hourly'
    print(url)
    r = requests.get(url)
    return r.json()

In [7]:
def get_raw_weather_forecast(office, gridX, gridY):
    url = f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}'
    print(url)
    r = requests.get(url)
    return r.json()

In [9]:
#forecast_7day_hourly

In [10]:
ct_forecast = get_weather_forecast(ct_grid, ct_grid_x, ct_grid_y)
ct_forecast

https://api.weather.gov/gridpoints/LOX/160,48/forecast


{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-118.1416245, 34.1484987],
    [-118.13699860000001, 34.1262836],
    [-118.11015800000001, 34.1301103],
    [-118.11477870000002, 34.1523259],
    [-118.1416245, 34.1484987]]]},
 'properties': {'updated': '2024-04-07T16:06:56+00:00',
  'units': 'us',
  'forecastGenerator': 'BaselineForecastGenerator',
  'generatedAt': '2024-04-07T16:12:27+00:00',
  'updateTime': '2024-04-07T16:06:56+00:00',
  'validTimes': '2024-04-07T10:00:00+00:00/P7DT15H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 227.076},
  'periods': [{'number': 1,
    'name': 'Today',
    'startTime': '2024-04-07T09:00:00-07:00',
    'endTime': '2024-04-07T18:00:0

In [11]:
def create_forecast_df(json_forecast):
    forecast = json_forecast['properties']['periods']
    forecast_df = pd.DataFrame(forecast)
    forecast_df['startTime'] = pd.to_datetime(forecast_df['startTime'], utc=True)
    forecast_df['endTime'] = pd.to_datetime(forecast_df['endTime'], utc=True)
    return forecast_df

In [12]:
ct_forecast_df = create_forecast_df(ct_forecast)
ct_forecast_df

,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast
0,1,Today,2024-04-07 16:00:00+00:00,2024-04-08 01:00:00+00:00,True,65,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.111111...","{'unitCode': 'wmoUnit:percent', 'value': 71}",5 to 10 mph,SSW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 65. South southwest wi..."
1,2,Tonight,2024-04-08 01:00:00+00:00,2024-04-08 13:00:00+00:00,False,45,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 5.555555...","{'unitCode': 'wmoUnit:percent', 'value': 80}",5 to 10 mph,ESE,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 45. East south..."
2,3,Monday,2024-04-08 13:00:00+00:00,2024-04-09 01:00:00+00:00,True,73,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 3.888888...","{'unitCode': 'wmoUnit:percent', 'value': 81}",5 to 15 mph,ENE,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 73. East northeast win..."
3,4,Monday Night,2024-04-09 01:00:00+00:00,2024-04-09 13:00:00+00:00,False,51,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 0.555555...","{'unitCode': 'wmoUnit:percent', 'value': 38}",5 to 10 mph,NNE,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 51. North northeast w..."
4,5,Tuesday,2024-04-09 13:00:00+00:00,2024-04-10 01:00:00+00:00,True,78,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 2.222222...","{'unitCode': 'wmoUnit:percent', 'value': 56}",5 to 15 mph,NNW,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 78. North northwest wi..."
5,6,Tuesday Night,2024-04-10 01:00:00+00:00,2024-04-10 13:00:00+00:00,False,54,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 3.888888...","{'unitCode': 'wmoUnit:percent', 'value': 52}",5 to 10 mph,NNE,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 54."
6,7,Wednesday,2024-04-10 13:00:00+00:00,2024-04-11 01:00:00+00:00,True,82,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.111111...","{'unitCode': 'wmoUnit:percent', 'value': 52}",5 to 10 mph,SSW,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 82."
7,8,Wednesday Night,2024-04-11 01:00:00+00:00,2024-04-11 13:00:00+00:00,False,57,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 8.333333...","{'unitCode': 'wmoUnit:percent', 'value': 64}",5 to 10 mph,NE,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 57."
8,9,Thursday,2024-04-11 13:00:00+00:00,2024-04-12 01:00:00+00:00,True,83,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 9.444444...","{'unitCode': 'wmoUnit:percent', 'value': 63}",5 to 10 mph,SW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 83."
9,10,Thursday Night,2024-04-12 01:00:00+00:00,2024-04-12 13:00:00+00:00,False,56,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 10.55555...","{'unitCode': 'wmoUnit:percent', 'value': 82}",5 to 10 mph,S,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,"Partly cloudy, with a low around 56."


In [13]:
ct_hourly_forecast = get_hourly_weather_forecast(ct_grid, ct_grid_x, ct_grid_y)
ct_hourly_forecast

https://api.weather.gov/gridpoints/LOX/160,48/forecast/hourly


{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-118.1416245, 34.1484987],
    [-118.13699860000001, 34.1262836],
    [-118.11015800000001, 34.1301103],
    [-118.11477870000002, 34.1523259],
    [-118.1416245, 34.1484987]]]},
 'properties': {'updated': '2024-04-07T16:06:56+00:00',
  'units': 'us',
  'forecastGenerator': 'HourlyForecastGenerator',
  'generatedAt': '2024-04-07T16:42:07+00:00',
  'updateTime': '2024-04-07T16:06:56+00:00',
  'validTimes': '2024-04-07T10:00:00+00:00/P7DT15H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 227.076},
  'periods': [{'number': 1,
    'name': '',
    'startTime': '2024-04-07T09:00:00-07:00',
    'endTime': '2024-04-07T10:00:00-07:00

In [14]:
import numpy as np
def create_hourly_forecast_df(json_forecast):
    # grab each hourly entry in the forecast
    forecast = json_forecast['properties']['periods']
    forecast_df = pd.DataFrame(forecast)
    
    # convert startTime to UTC datetime and set as index
    forecast_df['time'] = pd.to_datetime(forecast_df['startTime'], utc=True)
    forecast_df = forecast_df.set_index('time')
    
    # convert temperature to degF and degC
    forecast_df.rename(columns = {'temperature':'temperature_degF'}, inplace = True) 
    forecast_df['temperature_degC'] = forecast_df['temperature_degF'].apply(lambda x: np.round((x - 32) * 5.0/9.0, 2))
    
    # convert dewpoint temperature to degF and degC
    forecast_df['dewpoint_degC'] = forecast_df['dewpoint'].apply(lambda x: np.round(x['value'], 2))
    forecast_df['dewpoint_degF'] = forecast_df['dewpoint_degC'].apply(lambda x: np.round(x * 9.0/5.0 + 32, 2))
    
    # extract value from windSpeed column and convert to int
    forecast_df['windSpeed_mph'] = forecast_df['windSpeed'].apply(lambda x: int(x.split(" mph")[0]))
    
    # extract value from json formatted columns
    forecast_df['probabilityOfPrecipitationPercent'] = forecast_df['probabilityOfPrecipitation'].apply(lambda x: x['value'])
    forecast_df['relativeHumidityPercent'] = forecast_df['relativeHumidity'].apply(lambda x: x['value'])
    
    # drop columns that are no longer needed
    return forecast_df.drop(columns=['startTime', 'endTime', 'windSpeed', 'number', 'name', 'detailedForecast', 'dewpoint', 'probabilityOfPrecipitation', 'relativeHumidity', 'temperatureTrend', 'temperatureUnit'])

In [15]:
ct_hourly_forecast_df = create_hourly_forecast_df(ct_hourly_forecast)
ct_hourly_forecast_df

,isDaytime,temperature_degF,windDirection,icon,shortForecast,temperature_degC,dewpoint_degC,dewpoint_degF,windSpeed_mph,probabilityOfPrecipitationPercent,relativeHumidityPercent
time,,,,,,,,,,,
2024-04-07 16:00:00+00:00,True,49,SE,"https://api.weather.gov/icons/land/day/sct,0?s...",Mostly Sunny,9.44,4.44,39.99,5,0,71
2024-04-07 17:00:00+00:00,True,52,SE,"https://api.weather.gov/icons/land/day/sct,0?s...",Mostly Sunny,11.11,4.44,39.99,5,0,64
2024-04-07 18:00:00+00:00,True,57,SSW,"https://api.weather.gov/icons/land/day/few,1?s...",Sunny,13.89,5.56,42.01,5,1,57
2024-04-07 19:00:00+00:00,True,60,SSW,"https://api.weather.gov/icons/land/day/few,1?s...",Sunny,15.56,5.56,42.01,5,1,51
2024-04-07 20:00:00+00:00,True,63,SSW,"https://api.weather.gov/icons/land/day/few,1?s...",Sunny,17.22,5.56,42.01,5,1,47
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-13 23:00:00+00:00,True,68,SSW,"https://api.weather.gov/icons/land/day/sct,3?s...",Mostly Sunny,20.00,10.00,50.00,10,3,53
2024-04-14 00:00:00+00:00,True,67,SW,"https://api.weather.gov/icons/land/day/few,2?s...",Sunny,19.44,10.00,50.00,15,2,56
2024-04-14 01:00:00+00:00,False,65,SW,"https://api.weather.gov/icons/land/night/few,2...",Mostly Clear,18.33,10.00,50.00,15,2,59


In [16]:
jpl_forecast = get_weather_forecast(jpl_grid, jpl_grid_x, jpl_grid_y)
jpl_forecast_df = create_forecast_df(jpl_forecast)
jpl_forecast_df

https://api.weather.gov/gridpoints/LOX/159,51/forecast


,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast
0,1,Today,2024-04-07 16:00:00+00:00,2024-04-08 01:00:00+00:00,True,63,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.666666...","{'unitCode': 'wmoUnit:percent', 'value': 74}",5 to 10 mph,SSW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 63. South southwest wi..."
1,2,Tonight,2024-04-08 01:00:00+00:00,2024-04-08 13:00:00+00:00,False,44,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 5}","{'unitCode': 'wmoUnit:percent', 'value': 79}",5 to 10 mph,NW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 44. Northwest ..."
2,3,Monday,2024-04-08 13:00:00+00:00,2024-04-09 01:00:00+00:00,True,70,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 2.777777...","{'unitCode': 'wmoUnit:percent', 'value': 77}",5 to 15 mph,NNE,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 70. North northeast wi..."
3,4,Monday Night,2024-04-09 01:00:00+00:00,2024-04-09 13:00:00+00:00,False,51,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 3.333333...","{'unitCode': 'wmoUnit:percent', 'value': 46}",10 to 15 mph,NNE,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 51. North northeast w..."
4,5,Tuesday,2024-04-09 13:00:00+00:00,2024-04-10 01:00:00+00:00,True,76,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 2.777777...","{'unitCode': 'wmoUnit:percent', 'value': 54}",10 to 15 mph,N,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 76. North wind 10 to 1..."
5,6,Tuesday Night,2024-04-10 01:00:00+00:00,2024-04-10 13:00:00+00:00,False,55,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 2.222222...","{'unitCode': 'wmoUnit:percent', 'value': 39}",5 to 10 mph,NNE,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 55."
6,7,Wednesday,2024-04-10 13:00:00+00:00,2024-04-11 01:00:00+00:00,True,80,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.111111...","{'unitCode': 'wmoUnit:percent', 'value': 48}",5 to 10 mph,SSW,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 80."
7,8,Wednesday Night,2024-04-11 01:00:00+00:00,2024-04-11 13:00:00+00:00,False,57,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 7.222222...","{'unitCode': 'wmoUnit:percent', 'value': 59}",5 to 10 mph,NE,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 57."
8,9,Thursday,2024-04-11 13:00:00+00:00,2024-04-12 01:00:00+00:00,True,80,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 8.888888...","{'unitCode': 'wmoUnit:percent', 'value': 57}",5 to 10 mph,SSW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 80."
9,10,Thursday Night,2024-04-12 01:00:00+00:00,2024-04-12 13:00:00+00:00,False,55,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 9.444444...","{'unitCode': 'wmoUnit:percent', 'value': 73}",5 to 10 mph,S,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,"Partly cloudy, with a low around 55."


In [17]:
office_forecast = get_weather_forecast(office_grid, office_grid_x, office_grid_y)
office_forecast_df = create_forecast_df(office_forecast)
office_forecast_df

https://api.weather.gov/gridpoints/MTR/99,82/forecast


,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast
0,1,Today,2024-04-07 16:00:00+00:00,2024-04-08 01:00:00+00:00,True,63,F,falling,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.666666...","{'unitCode': 'wmoUnit:percent', 'value': 80}",6 to 14 mph,NW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny. High near 63, with temperatures falling..."
1,2,Tonight,2024-04-08 01:00:00+00:00,2024-04-08 13:00:00+00:00,False,41,F,rising,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.666666...","{'unitCode': 'wmoUnit:percent', 'value': 94}",5 to 14 mph,WNW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear. Low around 41, with temperatures..."
2,3,Monday,2024-04-08 13:00:00+00:00,2024-04-09 01:00:00+00:00,True,71,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.666666...","{'unitCode': 'wmoUnit:percent', 'value': 96}",5 to 13 mph,NW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 71. Northwest wind 5 t..."
3,4,Monday Night,2024-04-09 01:00:00+00:00,2024-04-09 13:00:00+00:00,False,45,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 7.777777...","{'unitCode': 'wmoUnit:percent', 'value': 83}",2 to 13 mph,NNW,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 45. North northwest w..."
4,5,Tuesday,2024-04-09 13:00:00+00:00,2024-04-10 01:00:00+00:00,True,77,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 8.888888...","{'unitCode': 'wmoUnit:percent', 'value': 83}",2 to 12 mph,ENE,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 77. East northeast win..."
5,6,Tuesday Night,2024-04-10 01:00:00+00:00,2024-04-10 13:00:00+00:00,False,50,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 10}","{'unitCode': 'wmoUnit:percent', 'value': 82}",3 to 12 mph,N,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 50."
6,7,Wednesday,2024-04-10 13:00:00+00:00,2024-04-11 01:00:00+00:00,True,81,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 12.22222...","{'unitCode': 'wmoUnit:percent', 'value': 82}",3 to 9 mph,NNE,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 81."
7,8,Wednesday Night,2024-04-11 01:00:00+00:00,2024-04-11 13:00:00+00:00,False,52,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 12.77777...","{'unitCode': 'wmoUnit:percent', 'value': 85}",3 to 9 mph,NNW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 52."
8,9,Thursday,2024-04-11 13:00:00+00:00,2024-04-12 01:00:00+00:00,True,82,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...","{'unitCode': 'wmoUnit:percent', 'value': 85}",3 to 12 mph,NNE,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 82."
9,10,Thursday Night,2024-04-12 01:00:00+00:00,2024-04-12 13:00:00+00:00,False,51,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...","{'unitCode': 'wmoUnit:percent', 'value': 90}",6 to 12 mph,WNW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 51."


# 4 put it all together

In [22]:
def get_processed_hourly_7day_weather(latitude:float, longitude:float, test_mode:bool=False) -> pd.DataFrame:
    """given the latitude and longitude of a site, return a processed dataframe of the hourly weather. This is a orchestrator function
    args:
        latitude: float - the latitude of the site
        longitude: float - the longitude of the site
        test_mode: bool - if test mode is true, pull stored data from disk, otherwise call the apis
    returns
        weather_df: pd.DataFrame - the processed dataframe of the hourly 7day weather for that site
    """
    if test_mode:
        return pd.DataFrame({})
    office, grid_x, grid_y = get_grid_points(latitude, longitude)
    weather_json = get_hourly_weather_forecast(office, grid_x, grid_y)
    weather_df = create_hourly_forecast_df(weather_json)
    return weather_df

In [23]:
# test 
get_processed_hourly_7day_weather(latitude=caltech_lat, longitude=caltech_lon, test_mode=False)

https://api.weather.gov/points/34.134785646454844,-118.11691382579643
status code: 200
https://api.weather.gov/gridpoints/LOX/160,48/forecast/hourly


,isDaytime,temperature_degF,windDirection,icon,shortForecast,temperature_degC,dewpoint_degC,dewpoint_degF,windSpeed_mph,probabilityOfPrecipitationPercent,relativeHumidityPercent
time,,,,,,,,,,,
2024-04-07 16:00:00+00:00,True,49,SE,"https://api.weather.gov/icons/land/day/sct,0?s...",Mostly Sunny,9.44,4.44,39.99,5,0,71
2024-04-07 17:00:00+00:00,True,52,SE,"https://api.weather.gov/icons/land/day/sct,0?s...",Mostly Sunny,11.11,4.44,39.99,5,0,64
2024-04-07 18:00:00+00:00,True,57,SSW,"https://api.weather.gov/icons/land/day/few,1?s...",Sunny,13.89,5.56,42.01,5,1,57
2024-04-07 19:00:00+00:00,True,60,SSW,"https://api.weather.gov/icons/land/day/few,1?s...",Sunny,15.56,5.56,42.01,5,1,51
2024-04-07 20:00:00+00:00,True,63,SSW,"https://api.weather.gov/icons/land/day/few,1?s...",Sunny,17.22,5.56,42.01,5,1,47
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-13 23:00:00+00:00,True,68,SSW,"https://api.weather.gov/icons/land/day/sct,3?s...",Mostly Sunny,20.00,10.00,50.00,10,3,53
2024-04-14 00:00:00+00:00,True,67,SW,"https://api.weather.gov/icons/land/day/few,2?s...",Sunny,19.44,10.00,50.00,15,2,56
2024-04-14 01:00:00+00:00,False,65,SW,"https://api.weather.gov/icons/land/night/few,2...",Mostly Clear,18.33,10.00,50.00,15,2,59
